# Data Cleaning Challenge: Cleaning numeric columns

[Original Version by Rachael Tatman](https://www.kaggle.com/rtatman/data-cleaning-challenge-cleaning-numeric-columns)

Welcome to Day 5 of the 5-Day Data Challenge! (Can you believe it's the last day already?) Today, we're going to be learning how to clean up columns with dates and numbers in them that R doesn’t realize are dates or numbers. In particular, we'll learn how to remove symbols like "%" or "$", and how to get R to correctly parse dates so you can do things like plot days in order. 

I'll start by introducing each concept or technique, and then you'll get a chance to apply it with an exercise (look for the **Your turn!** section). Ready? Let's get started!
___

**Kernel FAQs:**

* **How do I get started?**   To get started, click the blue "Fork Notebook" button in the upper, right hand corner. This will create a private copy of this notebook that you can edit and play with. Once you're finished with the exercises, you can choose to make your notebook public to share with others. :)

* **How do I run the code in this notebook?** Once you fork the notebook, it will open in the notebook editor. From there you can write code in any code cell (the ones with the grey background) and run the code by either 1) clicking in the code cell and then hitting CTRL + ENTER or 2) clicking in the code cell and the clicking on the white "play" arrow to the left of the cell. If you want to run all the code in your notebook, you can use the double, "fast forward" arrows at the bottom of the notebook editor.

* **How do I save my work?** Any changes you make are saved automatically as you work. You can run all the code in your notebook and save a static version by hitting the blue "Commit & Run" button in the upper right hand corner of the editor. 

* **How can I find my notebook again later?** The easiest way is to go to your user profile (https://www.kaggle.com/replace-this-with-your-username), then click on the "Kernels" tab. All of your kernels will be under the "Your Work" tab, and all the kernels you've upvoted will be under the "Favorites" tab.

___


# Get our environment set up
___

At this point you know the drill: we need to get our libraries and data all read in and ready to go. 

In [2]:
# libraries we'll need
library(tidyverse)
library(lubridate)
library(IRdisplay)
library(repr)

# read in the data we'll need
listings <- read_csv("boston-airbnb-open-data/listings.csv")
hospital_charges <- read_csv("inpatient-hospital-charges/inpatientCharges.csv") %>%
    janitor::clean_names() # clean up column names, with clean_names() from the janitor package
drone_strikes <- read_csv("pakistandroneattacks/PakistanDroneAttacksWithTemp Ver 9 (October 19, 2017).csv") %>%
    janitor::clean_names() # clean up column names
# remove the last row (has totals in it)
drone_strikes <- drone_strikes[-nrow(drone_strikes),] 
mass_shootings <- read_csv("us-mass-shootings-last-50-years/Mass Shootings Dataset Ver 5.csv")%>%
    janitor::clean_names() # clean up column names

Parsed with column specification:
cols(
  .default = col_character(),
  id = col_integer(),
  scrape_id = col_double(),
  last_scraped = col_date(format = ""),
  host_id = col_integer(),
  host_since = col_date(format = ""),
  host_listings_count = col_integer(),
  host_total_listings_count = col_integer(),
  latitude = col_double(),
  longitude = col_double(),
  accommodates = col_integer(),
  bathrooms = col_double(),
  bedrooms = col_integer(),
  beds = col_integer(),
  square_feet = col_integer(),
  guests_included = col_integer(),
  minimum_nights = col_integer(),
  maximum_nights = col_integer(),
  availability_30 = col_integer(),
  availability_60 = col_integer(),
  availability_90 = col_integer()
  # ... with 14 more columns
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  `DRG Definition` = col_character(),
  `Provider Id` = col_integer(),
  `Provider Name` = col_character(),
  `Provider Street Address` = col_character(),
  `Provider Ci

In [3]:
display_html("<style> 
h1, h2, h3 {text-shadow: 4px 4px 4px #aaa;} 
span {color: black; text-shadow: 4px 4px 4px #aaa;}
div.output_prompt {color: darkblue;} 
div.input_prompt {color: steelblue;} 
</style>")

# Removing non-numeric characters
___

Sometimes you'll get a dataset and find that someone has helpfully added percentage signs, dollar signs or other non-numeric symbols that tell you the units of the data you're working with. While this is great when you're looking at the data, it can lead to problems when you're trying to work with it in R. This is because when you read in a dataframe in R, it makes a guess about what data type each column is by looking at what is in that column. And if it runs in to any characters that aren't numbers in your column, it will play it safe and say that the datatype is "character" rather than "numeric". This means that in order to actually do any math with that column, you need some way to remove any non-numeric characters.

If you have tried to solve this problem in the past, you may have come across people suggesting that you use regular expressions. While I'm a big fan of regular expressions (I use one to get days of the week later on in this kernel), I would *strongly* advise you to steer clear of them in data cleaning unless you have no other choice. Why?

* **Regular expressions are brittle.** Since they rely on matching a very specific set of characters in a specific order, if you get new data later that's slightly different, your regular expressions won't work. And if you haven't had to debug regular expressions previously, count yourself lucky: it's a huge pain.
* **Regular expressions are hard to write & read.** If you've worked with regular expressions before, you know that they are hard to get right! They're also very hard to read: can you tell what this is supposed at first glance? `^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$`.  (It's [a Roman Numeral finder](https://stackoverflow.com/a/800868/9403317), in case you were wondering.) If something is both hard to write and hard to read that means you and your team will spend more time on it than you need to, especially if there's another option.

Instead, the option I'd recommend is `parse_numeric()` from the readr package by Jim Hester. This is one of the things we load in with `library(tidyverse)`. `parse_numeric()` can handle most of the common ways that numbers are presented and neatly tidies up the non-numeric characters and changes the class of the resulting object:

In [4]:
# character vector of numbers
to_parse <- c(100, "10,000", "%100", "$50")

# check to make sure it's numeric
print("Class before:")
class(to_parse)

# parse numbers
parsed_numbers <- parse_number(to_parse)

# check class
print("Class after:")
class(parsed_numbers)

# see what it looks like now
parsed_numbers

[1] "Class before:"


[1] "character"

[1] "Class after:"


[1] "numeric"

[1]   100 10000   100    50

Of  course, it's easy to use with a toy example, by how well does it extend to real datasets? Pretty well, in my experience! 

First, I like to pull out all the columns that are "character" class and look at them. (I focus on character columns because I don't need to re-parse the numeric columns that were parsed correctly the first time.)

In [5]:
# get only columns with the data type "character" 
character_columns <- hospital_charges[, sapply(hospital_charges, class) == "character"]

# look at these columns
str(character_columns)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	163065 obs. of  9 variables:
 $ drg_definition                      : chr  "039 - EXTRACRANIAL PROCEDURES W/O CC/MCC" "039 - EXTRACRANIAL PROCEDURES W/O CC/MCC" "039 - EXTRACRANIAL PROCEDURES W/O CC/MCC" "039 - EXTRACRANIAL PROCEDURES W/O CC/MCC" ...
 $ provider_name                       : chr  "SOUTHEAST ALABAMA MEDICAL CENTER" "MARSHALL MEDICAL CENTER SOUTH" "ELIZA COFFEE MEMORIAL HOSPITAL" "ST VINCENT'S EAST" ...
 $ provider_street_address             : chr  "1108 ROSS CLARK CIRCLE" "2505 U S HIGHWAY 431 NORTH" "205 MARENGO STREET" "50 MEDICAL PARK EAST DRIVE" ...
 $ provider_city                       : chr  "DOTHAN" "BOAZ" "FLORENCE" "BIRMINGHAM" ...
 $ provider_state                      : chr  "AL" "AL" "AL" "AL" ...
 $ hospital_referral_region_description: chr  "AL - Dothan" "AL - Birmingham" "AL - Birmingham" "AL - Birmingham" ...
 $ average_covered_charges             : chr  "$32963.07" "$15131.85" "$37560.37" "$13998.28" ...
 $ aver

Looking at these columns, it seems like only the last three have been parsed incorrectly: each of them should actually be numeric. My next step is to select just those columns and parse each of them. 

In [6]:
# select columns with "charge" or "pay" in the name
money_columns <- character_columns %>%
    select(contains("charge"), contains("pay")) 

# parse each of those columns as numeric using sapply()
money_columns_parsed <- sapply(money_columns, parse_number) %>%
    as_data_frame()

Finally, I make a new copy of the dataset with the parsed columns. (You may want to avoid making a bunch of copies like this if your dataset is very large, but I like the safety net of not modifying data in place.)

In [7]:
# replace columns with their parsed versions
hospital_charges_parsed <- hospital_charges %>%
    # the next line *removes* the columns we selected earlier
    select(-contains("charge"), -contains("pay")) %>%
    # add the columns we parsed earlier
    bind_cols(money_columns_parsed)

# double check that our data types are correct
str(hospital_charges_parsed)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	163065 obs. of  11 variables:
 $ drg_definition                      : chr  "039 - EXTRACRANIAL PROCEDURES W/O CC/MCC" "039 - EXTRACRANIAL PROCEDURES W/O CC/MCC" "039 - EXTRACRANIAL PROCEDURES W/O CC/MCC" "039 - EXTRACRANIAL PROCEDURES W/O CC/MCC" ...
 $ provider_id                         : int  10001 10005 10006 10011 10016 10023 10029 10033 10039 10040 ...
 $ provider_name                       : chr  "SOUTHEAST ALABAMA MEDICAL CENTER" "MARSHALL MEDICAL CENTER SOUTH" "ELIZA COFFEE MEMORIAL HOSPITAL" "ST VINCENT'S EAST" ...
 $ provider_street_address             : chr  "1108 ROSS CLARK CIRCLE" "2505 U S HIGHWAY 431 NORTH" "205 MARENGO STREET" "50 MEDICAL PARK EAST DRIVE" ...
 $ provider_city                       : chr  "DOTHAN" "BOAZ" "FLORENCE" "BIRMINGHAM" ...
 $ provider_state                      : chr  "AL" "AL" "AL" "AL" ...
 $ provider_zip_code                   : int  36301 35957 35631 35235 35007 36116 36801 35233 35801 35903 ...
 $

And now our numeric columns are actually numeric! From here you can work with them as you would any other numeric values.

## Your turn!
___

Take a look at the `listings` dataset, which contains information on Airbnb listings. You should find quite a few numeric columns read in as characters that need to be parsed. If you're looking for more of a challenge, there are also some logical columns (with 't' and 'f') that have been parsed as characters. Try parsing them using `parse_logical()`. 

If you're looking an extra-tough challenge, try writing a function that identifies which numeric columns might have been mis-parsed as characters and attempts to parse them correctly.  

In [9]:
# your code here :)
# get only columns with the data type "character" 
character_columns_listings <- listings[, sapply(listings, class) == "character"]

# look at these columns
str(character_columns_listings)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	3585 obs. of  61 variables:
 $ listing_url                     : chr  "https://www.airbnb.com/rooms/12147973" "https://www.airbnb.com/rooms/3075044" "https://www.airbnb.com/rooms/6976" "https://www.airbnb.com/rooms/1436513" ...
 $ name                            : chr  "Sunny Bungalow in the City" "Charming room in pet friendly apt" "Mexican Folk Art Haven in Boston" "Spacious Sunny Bedroom Suite in Historic Home" ...
 $ summary                         : chr  "Cozy, sunny, family home.  Master bedroom high ceilings. Deck, garden with hens, beehives & play structure.   S"| __truncated__ "Charming and quiet room in a second floor 1910 condo building. The room has a full size bed, darkening curtains"| __truncated__ "Come stay with a friendly, middle-aged guy in the safe and quiet Roslindale neighborhood of Boston. You will ha"| __truncated__ "Come experience the comforts of home away from home in our fabulous bedroom suite available in Roslindale,

In [10]:
# select columns with logical values "f"/"t"
logical_columns_listings <- character_columns_listings %>%
    select(contains("host_is"), 
           contains("host_has"), 
           contains("host_identity"), 
           contains("is_location"), 
           contains("require"), 
           contains("bookable")) 

# parse each of those columns as logical using sapply()
logical_columns_listings_parsed <- sapply(logical_columns_listings, parse_logical) %>%
    as_data_frame()
str(logical_columns_listings_parsed)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	3585 obs. of  8 variables:
 $ host_is_superhost               : logi  FALSE FALSE TRUE FALSE TRUE TRUE ...
 $ host_has_profile_pic            : logi  TRUE TRUE TRUE TRUE TRUE TRUE ...
 $ host_identity_verified          : logi  FALSE TRUE TRUE FALSE TRUE TRUE ...
 $ is_location_exact               : logi  TRUE TRUE TRUE FALSE TRUE TRUE ...
 $ requires_license                : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ require_guest_profile_picture   : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ require_guest_phone_verification: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ instant_bookable                : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...


In [11]:
# select columns with "price", "extra_people", "fee", "deposit" in the name
money_columns_listings <- character_columns_listings %>%
    select(contains("price"),
           contains("extra_people"),
           contains("fee"),
           contains("deposit")) 

# parse each of those columns as numeric using sapply()
money_columns_listings_parsed <- sapply(money_columns_listings, parse_number) %>%
    as_data_frame()

In [12]:
head(money_columns_listings_parsed)

price,weekly_price,monthly_price,extra_people,cleaning_fee,security_deposit
250,NA,NA,0,35,NA
65,400,NA,0,10,95
65,395,1350,20,NA,NA
75,NA,NA,25,50,100
79,NA,NA,0,15,NA
75,NA,NA,0,30,NA


In [14]:
# replace columns with their parsed versions
listings_parsed1 <- listings %>%
    # the next line *removes* the columns we selected earlier
    select(-contains("host_is"),
           -contains("host_has"),
           -contains("host_identity"),
           -contains("is_location"),
           -contains("require"),
           -contains("bookable")) %>%
    # add the columns we parsed earlier
    bind_cols(logical_columns_listings_parsed)

logical_columns <- listings_parsed1[, sapply(listings_parsed1, class) == "logical"]
# double check that our data types are correct
str(logical_columns)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	3585 obs. of  8 variables:
 $ host_is_superhost               : logi  FALSE FALSE TRUE FALSE TRUE TRUE ...
 $ host_has_profile_pic            : logi  TRUE TRUE TRUE TRUE TRUE TRUE ...
 $ host_identity_verified          : logi  FALSE TRUE TRUE FALSE TRUE TRUE ...
 $ is_location_exact               : logi  TRUE TRUE TRUE FALSE TRUE TRUE ...
 $ requires_license                : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ require_guest_profile_picture   : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ require_guest_phone_verification: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ instant_bookable                : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...


In [15]:
# replace columns with their parsed versions
listings_parsed2 <- listings_parsed1 %>%
    # the next line *removes* the columns we selected earlier
    select(-contains("price"),
           -contains("extra_people"),
           -contains("fee"),
           -contains("deposit")) %>%
    # add the columns we parsed earlier
    bind_cols(money_columns_listings_parsed)

numeric_columns <- listings_parsed2[, sapply(listings_parsed2, class) == "numeric"]
# double check that our data types are correct
str(numeric_columns)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	3585 obs. of  11 variables:
 $ scrape_id        : num  2.02e+13 2.02e+13 2.02e+13 2.02e+13 2.02e+13 ...
 $ latitude         : num  42.3 42.3 42.3 42.3 42.3 ...
 $ longitude        : num  -71.1 -71.1 -71.1 -71.1 -71.1 ...
 $ bathrooms        : num  1.5 1 1 1 1.5 1 1 2 1 1 ...
 $ reviews_per_month: num  NA 1.3 0.47 1 2.25 1.7 4 2.38 5.36 1.01 ...
 $ price            : num  250 65 65 75 79 75 100 75 58 229 ...
 $ weekly_price     : num  NA 400 395 NA NA NA NA NA NA NA ...
 $ monthly_price    : num  NA NA 1350 NA NA NA NA NA NA NA ...
 $ extra_people     : num  0 0 20 25 0 0 25 15 0 25 ...
 $ cleaning_fee     : num  35 10 NA 50 15 30 NA 10 NA 50 ...
 $ security_deposit : num  NA 95 NA 100 NA NA NA NA NA 200 ...


# Parsing dates & times
___

Sometimes you're lucky and your dates will be in a format that R knows how to handle and will be read in and parsed automatically. Sometimes, however, you'll have to let R know that a certain column contains a date and what the format of that date is. This is known as *parsing* a date. I generally use [the lubridate package](https://cran.r-project.org/web/packages/lubridate/index.html) by Vitalie Spinu and co-authors for parsing dates. These are the four functions I use most often to parse dates in different formats:

* **mdy()**, for dates that are month, day and then year
* **ymd()**, for dates that are year, month and then day
*  **dmy()**, for dates that are day, month and then year
* **ymd_hms()**, for dates that are year, month, day, hour, minute and finally second

Once a date is parsed, you can easily extract parts of it using functions like month() and year() and plot or analyze it like any other numeric variable. You can learn more about parsing dates [here](https://cran.r-project.org/web/packages/lubridate/vignettes/lubridate.html).

Now that we've got the basics, let's put them into practice. Here are some of the dates in the date column of the `drone_strikes` dataset. 

In [16]:
# look at the dates
drone_strikes$date[1:10]

[1] "Friday, June 18, 2004"       "Sunday, May 08, 2005"       
 [3] "Thursday, December 01, 2005" "Friday, January 06, 2006"   
 [5] "Friday, January 13, 2006"    "Monday, October 30, 2006"   
 [7] "Friday, April 27, 2007"      "Tuesday, June 19, 2007"     
 [9] "Friday, November 02, 2007"   "Monday, December 03, 2007"

So our dates are in the format: day of week, month, day of month, year. Unfortunately, there's no lubridate parsing function for handling the day of the week, so we'll need to strip them. (Don't worry, once our dates are parsed we can get them all back with the `wday()` function.) Here, I've written a function to remove them for us:

In [17]:
# function to remove the day of the week (friday, monday, etc.) & following comma
remove_dow <- function(column){
    no_dow <- str_replace_all(column, '[A-Za-z]*day, ', '')
    return(no_dow)
}

# test it out
remove_dow(drone_strikes$date[1:10])

[1] "June 18, 2004"     "May 08, 2005"      "December 01, 2005"
 [4] "January 06, 2006"  "January 13, 2006"  "October 30, 2006" 
 [7] "April 27, 2007"    "June 19, 2007"     "November 02, 2007"
[10] "December 03, 2007"

Now we can use this function to remove our day of the week, then parse the remaining data with the `mdy()` function. If you don't need to remove the days of the week, you can just skip that line.

In [19]:
# tidy up dates & convert them to date format
dates <- drone_strikes %>%
    select(date) %>% # get the "date" column
    mutate(date_formatted = remove_dow(date)) %>% # remove the day of the week
    mutate(date_formatted = mdy(date_formatted)) # convert to date format

# compare before & after formatting
head(dates)

# add formatted dates to our dataframe
drone_strikes$date_formatted <- dates$date_formatted

date,date_formatted
"Friday, June 18, 2004",2004-06-18
"Sunday, May 08, 2005",2005-05-08
"Thursday, December 01, 2005",2005-12-01
"Friday, January 06, 2006",2006-01-06
"Friday, January 13, 2006",2006-01-13
"Monday, October 30, 2006",2006-10-30


And that's all there is to it! You're now ready to start your time series analysis or easily plot your dates using ggplot.

## Your turn!
___

Correctly parse the "date" column from the mass_shootings dataset. Print some of the unconverted & converted dates using the head() function to compare them.

In [20]:
# your code goes here :)
# look at the dates
t(head(mass_shootings$date))

11/5/2017,11/1/2017,10/18/2017,10/1/2017,6/14/2017,6/7/2017


In [22]:
# convert dates to date format
formatted_date <- as.Date(mass_shootings$date, format = "%m/%d/%Y")
# add formatted dates to our dataframe
mass_shootings$formatted_date <- formatted_date
# look at the formatted dates
t(head(mass_shootings$formatted_date))

[1] "2017-11-05" "2017-11-01" "2017-10-18" "2017-10-01" "2017-06-14"
[6] "2017-06-07"

# And that's it, you've completed the whole challenge!
____

Congrats, you've finished the entire 5-Day Challenge on data cleaning with R! I hope you learned lots of helpful tips and tricks that you can apply in your work further down the line. :)

If you're looking for more content, try checking out [the other 5-Day Challenges](https://www.kaggle.com/rtatman/list-of-5-day-challenges/) or some of the great content we have on the [Kaggle Learn page](https://www.kaggle.com/learn/overview). If you're looking for more practice with data cleaning specifically, I'd recommend picking a new dataset ([maybe one of these](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=all&filetype=all&license=all&tagids=13202)), getting to know it (mainly doing a lot of visualizations) and then making the changes you need to use it for modelling or analysis. 